In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data.sampler import BatchSampler

In [2]:
device = torch.device('cpu')

In [3]:
input_size = 784 # 28x28
hidden_size = 500 
num_classes = 10
num_epochs = 2
batch_size = 100
#test_batch_size = 10000
learning_rate = 0.001
spc=1000
#SPC is for number of samples per class
t1=[]
t2=[]
t3=[]
i=0

In [4]:
# MNIST dataset 

train_dataset = torchvision.datasets.FashionMNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))]),  
                                           download=True)

test_dataset = torchvision.datasets.FashionMNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))]))


for i in range (10):
    t0=(train_dataset.targets==i).nonzero(as_tuple=False)
    t0 = t0.reshape(1, -1)
    t0 = t0.squeeze()
    t0=t0.tolist()
    np.random.shuffle(t0)
    t0=t0[0:spc]
    t1=t1+t0
    print ( "Current list size is" ,len(t1))
    print (spc," random samples for class" ,i, "is added to the list")

train_subset = torch.utils.data.Subset(train_dataset,t1)
#train_dataset0.targets = train_dataset.targets[t1]
#train_dataset0.data = train_dataset.data[t1]

t0=test_dataset.targets
t0 = t0.reshape(1, -1)
t0 = t0.squeeze()
t2=t0.tolist()
print ( "Test list size is" ,len(t2))

Current list size is 1000
1000  random samples for class 0 is added to the list
Current list size is 2000
1000  random samples for class 1 is added to the list
Current list size is 3000
1000  random samples for class 2 is added to the list
Current list size is 4000
1000  random samples for class 3 is added to the list
Current list size is 5000
1000  random samples for class 4 is added to the list
Current list size is 6000
1000  random samples for class 5 is added to the list
Current list size is 7000
1000  random samples for class 6 is added to the list
Current list size is 8000
1000  random samples for class 7 is added to the list
Current list size is 9000
1000  random samples for class 8 is added to the list
Current list size is 10000
1000  random samples for class 9 is added to the list
Test list size is 10000


In [5]:
# Data loader


train_loader = DataLoader(dataset=train_subset, batch_size=batch_size, shuffle=False)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size ,shuffle=False)

print (len(test_loader))
print (len(test_dataset))



100
10000


In [6]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = torch.nn.Linear(input_size, hidden_size) 
        self.relu = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out

In [7]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


In [9]:
# Train the model for all elements of the dataset
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


Epoch [1/2], Step [100/100], Loss: 0.6564
Epoch [2/2], Step [100/100], Loss: 0.1131


In [10]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
t3=[]
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        tpredict=predicted.tolist()
        t3=t3+tpredict
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        print("Current total samples are",n_samples,)
        print("of which ",n_correct,"samples are correct")

        
print(len(t3))
       
acc = 100.0 * n_correct / n_samples
print(f'Accuracy of the network on the given test images: {acc} %')


Current total samples are 100
of which  16 samples are correct
Current total samples are 200
of which  32 samples are correct
Current total samples are 300
of which  49 samples are correct
Current total samples are 400
of which  62 samples are correct
Current total samples are 500
of which  83 samples are correct
Current total samples are 600
of which  104 samples are correct
Current total samples are 700
of which  121 samples are correct
Current total samples are 800
of which  143 samples are correct
Current total samples are 900
of which  160 samples are correct
Current total samples are 1000
of which  171 samples are correct
Current total samples are 1100
of which  190 samples are correct
Current total samples are 1200
of which  205 samples are correct
Current total samples are 1300
of which  219 samples are correct
Current total samples are 1400
of which  234 samples are correct
Current total samples are 1500
of which  249 samples are correct
Current total samples are 1600
of which

In [11]:
from sklearn.metrics import classification_report
#print(classification_report(labels, predicted, zero_division=1))
print(classification_report(t2, t3, zero_division=1))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1000
           1       1.00      0.00      0.00      1000
           2       1.00      0.00      0.00      1000
           3       1.00      0.00      0.00      1000
           4       0.00      0.00      0.00      1000
           5       1.00      0.00      0.00      1000
           6       0.07      0.05      0.06      1000
           7       0.00      0.00      0.00      1000
           8       0.15      0.80      0.25      1000
           9       0.30      0.99      0.47      1000

    accuracy                           0.18     10000
   macro avg       0.45      0.18      0.08     10000
weighted avg       0.45      0.18      0.08     10000



In [12]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(t2,t3))

[[   0    0    0    0    0    0  347    4  623   26]
 [   0    0    0    0  341    0  111  120  425    3]
 [   0    0    0    0    0    0   37    0  958    5]
 [   0    0    0    0   22    0  190    2  762   24]
 [   0    0    0    0    0    0    7    0  988    5]
 [   0    0    0    0    0    0    0    0    6  994]
 [   0    0    0    0    0    0   52    1  924   23]
 [   0    0    0    0    0    0    0    0    0 1000]
 [   0    0    0    0    0    0    0    0  804  196]
 [   1    0    0    0    0    0    0    0    5  994]]
